In [1]:
# use conda enviroment "general" to run this script

import csv
import numpy as np



import sys

# ask Python to look for other scripts but not in the default directory
sys.path.append('../1_generate candidate sequences/scripts/') # The sys.path is a list that contains directories where Python looks for modules and packages
from oPool_regions import *

#print(sys.path)

In [2]:
selected_sequences = '../4_pick sequences/selected_sequences.csv'



portions_of_oPools = {}

with open(selected_sequences, 'r') as file:

    csv_reader = csv.reader(file)
    
    # Skip the header row
    next(csv_reader)
    
    for row in csv_reader:
                
        # row[0], which is the first column of the CSV file, is the portion of the oPool
        # row[1], which is the second column of the CSV file, is the replicate of the oPool
        # row[2], which is the third column of the CSV file, is the portions of the sequence
        
        name = row[0] + "_" + row[1]
        portions = row[2]
                
        portions = eval(portions) # convert string representation of list to actual list
        
        portions_of_oPools_2 = portions[:6]
        portions_of_oPools_3 = portions[6:]
        
        portions_of_oPools[name] = {'portions_of_oPools_2': portions_of_oPools_2, 
                                    'portions_of_oPools_3': portions_of_oPools_3}

In [3]:
def extract_from_CSV_file(CSV_file):
    
    dict = {}
    
    with open(CSV_file, 'r') as file:
    
        csv_reader = csv.reader(file)
        
        # Skip the header row
        next(csv_reader)
        
        for row in csv_reader:
            
            # the type of row is list
            
            # row[0], which is the first column of the CSV file, is the name of the oPool
            # row[1], which is the second column of the CSV file, is the sequence of the oPool
            # row[2], which is the third column of the CSV file, is the name of the sequence
            
            name_of_oPool = row[0]
            sequence_name = row[2]
            
            portions = []
            
            if name_of_oPool.find('oPools_2') != -1: # "!= -1" means that the substring is found
                
                for key, items in portions_of_oPools.items():
                    
                    if key == sequence_name:
                        
                        portions = items['portions_of_oPools_2']
            
            if name_of_oPool.find('oPools_3') != -1:
                
                if sequence_name == '10%_25%_#1':
                    
                    portions = [0, 0, 0, 0, 0, 0]
                
                else:
                
                    for key, items in portions_of_oPools.items():
                        
                        if key == sequence_name:
                            
                            portions = items['portions_of_oPools_3']
            
            
            
            dict[row[2]] = {'sequence': row[1], 
                            'portions': portions}
    
    return dict

In [4]:
oPools_1 = extract_from_CSV_file('../5_remove UG in upstream ELK1 3 prime UTR/ELK1_oPools_1.csv')
oPools_2 = extract_from_CSV_file('../5_remove UG in upstream ELK1 3 prime UTR/ELK1_oPools_2.csv')
oPools_3 = extract_from_CSV_file('../5_remove UG in upstream ELK1 3 prime UTR/ELK1_oPools_3.csv')

In [5]:
potential_combinations = {}



# add the wild type sequence of insert, which is before replacing any UG

wild_type = 'wild_type---0%_#1---0%_#1'
wild_type_insert = 'CAAAAGACTCCTGGGCTCACCTGTTAGCGCTGGCCCAGCCCAGGCCTTGGGACCTGGGGGTTGGTGATTTGGGGGACAGTGCTACACTCGTCTCCACTGTTTGTTTTACTTCCCCAAAATGGACCTTTTTTTTTTCTAAAGAGTCCCAGAGAATGGGGAATTGTTCCTGTAAATATATATTTTTCAAAGTGATGCTGGAGGCTGCTGGCACTTTTTCTCTGATGTTGTCAGGACTAGCTGAGCGTGGCACGCATGACTGTGTGTGGAGGGTGTGAGGCAGAGGTCGATTGTGAGAGAAACGGTGTGATGGCGTGTGTGAGAAACAATAGTAGGGACTGCATATCGGTAACCATGTGTGTGTCAGAGACGTGGAAAGCTGTGATTGTGTGTGAGAGTGTGCAGAAGTTGTGACCACAGACTTTTGATGGTGTGTCTATGTGGGAAATTGTGGCTGTGTTTCAGAAAATAACTGAGCCTGGATGAATAGTCAAGGTACTGTACAAGAACCTGTGATGGTATGCACAAAATAATGTCTACGAGAAGGGAACGGGGACTGCAGCCCT'

UG_amount = wild_type_insert.count("TG")

potential_combinations[wild_type] = {'sequence': wild_type_insert, 'UG_amount': UG_amount}

In [6]:
UG_amount_in_each_region = np.array([4, 3, 2, 7, 2, 6, 4, 10, 6, 4, 3, 4])



for sequence_name_of_oPools_1, sequence_of_oPools_1 in oPools_1.items():
    
    sequence_of_oPools_1 = sequence_of_oPools_1['sequence']
    
    for sequence_name_of_oPools_2, items_of_oPools_2 in oPools_2.items():
        
        sequence_of_oPools_2 = items_of_oPools_2['sequence']
        portions_of_oPools_2 = items_of_oPools_2['portions']
        
        for sequence_name_of_oPools_3, items_of_oPools_3 in oPools_3.items():
            
            sequence_of_oPools_3 = items_of_oPools_3['sequence']
            portions_of_oPools_3 = items_of_oPools_3['portions']
            
            
            
            name = sequence_name_of_oPools_1 + "---" + sequence_name_of_oPools_2 + "---" + sequence_name_of_oPools_3
            combination = sequence_of_oPools_1.replace(Gibson_assembly_overlapping_oPools_1_2, '') + sequence_of_oPools_2.replace(Gibson_assembly_overlapping_oPools_2_3, '') + sequence_of_oPools_3
            #portions = np.array(portions_of_oPools_2 + portions_of_oPools_3)
            
            
            
            # I know that it is confusing: 
            # some name is including "200%", which means that instead of removing UG, I add more UGs to the sequence
            # so cannot calculate the portion of these combinations
            
            
            
            # total UG amount in the whole insert sequence (oPools_1 + oPools_2 + oPools_3) is 76
            # total UG amount in the 12 UG-rich regions is 55
            
            
            
            UG_amount = combination.count("TG")
            
            
            
            potential_combinations[name] = {'sequence': combination, 
                                            'UG_amount': UG_amount}

In [7]:
def write_fasta_file(file_path, dict):
    
    with open(file_path, 'w') as fasta_file:
        
        for key, value in dict.items():
        
            fasta_file.write(f'>{key}\n')
            fasta_file.write(f'{value["sequence"]}\n')



write_fasta_file('potential_combination_of_inserts.fasta', potential_combinations)

In [8]:
def write_csv_file(file_path, dict):
    
    with open(file_path, 'w', newline='') as csv_file:
        
        writer = csv.writer(csv_file)
        
        # Write the header row
        writer.writerow(['insert', 'UG_amount', 'sequence'])
        
        for key, value in dict.items():
            writer.writerow([key, value['UG_amount'], value['sequence']])



write_csv_file('CSV_file_of_potential_combination_of_inserts.csv', potential_combinations)